# Analyze Hyperparameter Tuning Job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [1]:
!conda install --yes bokeh # the flag --yes is nescessary to continue the installation when being asked to continue the installation

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done


In [2]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client("sagemaker")

## You must have already run a hyperparameter tuning job to analyze it here.
## The Hyperparameter tuning jobs you have run are listed in the Training section on your SageMaker dashboard.
## Copy the name of a completed job you want to analyze from that list.
## For example: tuning_job_name = 'mxnet-training-201007-0054'.
tuning_job_name = "xgb-randsearch-20221130-14-58-38"

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [3]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

5 training jobs have completed


In [4]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2022, 11, 30, 14, 58, 47, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7736070156097412},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2022, 11, 30, 15, 7, 44, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:103721820087:training-job/xgb-randsearch-20221130-14-58-38-003-28388a96',
 'TrainingJobName': 'xgb-randsearch-20221130-14-58-38-003-28388a96',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2022, 11, 30, 15, 1, 1, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '3.332208090525499',
                          'colsample_bytree': '0.8797123421763627',
                          'lambda': '3.8018010898739276',
                          'max_depth': '8'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [5]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df.sort_values(by="FinalObjectiveValue")

Number of training jobs with valid objective: 5
{'lowest': 0.7453140020370483, 'highest': 0.7736070156097412}


,alpha,colsample_bytree,lambda,max_depth,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
4,0.020224,0.671193,0.012518,2.0,xgb-randsearch-20221130-14-58-38-001-d9ccad0c,Completed,0.745314,2022-11-30 15:00:42+00:00,2022-11-30 15:02:50+00:00,128.0
1,3.772438,0.794501,0.011864,2.0,xgb-randsearch-20221130-14-58-38-004-cebd133c,Completed,0.749182,2022-11-30 15:00:39+00:00,2022-11-30 15:02:47+00:00,128.0
3,0.284830,0.887194,0.507214,10.0,xgb-randsearch-20221130-14-58-38-002-a7d94c0b,Completed,0.767784,2022-11-30 15:01:09+00:00,2022-11-30 15:03:06+00:00,117.0
0,0.066060,0.975312,9.242000,11.0,xgb-randsearch-20221130-14-58-38-005-b941e49d,Completed,0.771590,2022-11-30 15:00:43+00:00,2022-11-30 15:02:36+00:00,113.0
2,3.332208,0.879712,3.801801,8.0,xgb-randsearch-20221130-14-58-38-003-28388a96,Completed,0.773607,2022-11-30 15:01:01+00:00,2022-11-30 15:07:44+00:00,403.0


In [6]:
# convert start time to timestamp
import time
import datetime
s = "01/12/2022"
time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())

1669852800.0

In [7]:
df.sort_values(by='TrainingStartTime')

,alpha,colsample_bytree,lambda,max_depth,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,3.772438,0.794501,0.011864,2.0,xgb-randsearch-20221130-14-58-38-004-cebd133c,Completed,0.749182,2022-11-30 15:00:39+00:00,2022-11-30 15:02:47+00:00,128.0
4,0.020224,0.671193,0.012518,2.0,xgb-randsearch-20221130-14-58-38-001-d9ccad0c,Completed,0.745314,2022-11-30 15:00:42+00:00,2022-11-30 15:02:50+00:00,128.0
0,0.066060,0.975312,9.242000,11.0,xgb-randsearch-20221130-14-58-38-005-b941e49d,Completed,0.771590,2022-11-30 15:00:43+00:00,2022-11-30 15:02:36+00:00,113.0
2,3.332208,0.879712,3.801801,8.0,xgb-randsearch-20221130-14-58-38-003-28388a96,Completed,0.773607,2022-11-30 15:01:01+00:00,2022-11-30 15:07:44+00:00,403.0
3,0.284830,0.887194,0.507214,10.0,xgb-randsearch-20221130-14-58-38-002-a7d94c0b,Completed,0.767784,2022-11-30 15:01:09+00:00,2022-11-30 15:03:06+00:00,117.0


In [8]:
# add colum with start time referenced to the beginning of the training as time delta in seconds
start_time = df['TrainingStartTime'].min()
df['TrainingStartTimeInSeconds'] = df['TrainingStartTime'].apply(lambda x: (x - start_time).total_seconds())

In [9]:
df.sort_values(by='TrainingStartTime')

,alpha,colsample_bytree,lambda,max_depth,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds,TrainingStartTimeInSeconds
1,3.772438,0.794501,0.011864,2.0,xgb-randsearch-20221130-14-58-38-004-cebd133c,Completed,0.749182,2022-11-30 15:00:39+00:00,2022-11-30 15:02:47+00:00,128.0,0.0
4,0.020224,0.671193,0.012518,2.0,xgb-randsearch-20221130-14-58-38-001-d9ccad0c,Completed,0.745314,2022-11-30 15:00:42+00:00,2022-11-30 15:02:50+00:00,128.0,3.0
0,0.066060,0.975312,9.242000,11.0,xgb-randsearch-20221130-14-58-38-005-b941e49d,Completed,0.771590,2022-11-30 15:00:43+00:00,2022-11-30 15:02:36+00:00,113.0,4.0
2,3.332208,0.879712,3.801801,8.0,xgb-randsearch-20221130-14-58-38-003-28388a96,Completed,0.773607,2022-11-30 15:01:01+00:00,2022-11-30 15:07:44+00:00,403.0,22.0
3,0.284830,0.887194,0.507214,10.0,xgb-randsearch-20221130-14-58-38-002-a7d94c0b,Completed,0.767784,2022-11-30 15:01:09+00:00,2022-11-30 15:03:06+00:00,117.0,30.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses. For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance exploration of new areas of parameter space against exploitation of known good areas. This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [14]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuner)

#p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
#p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")

p = figure(plot_width=900, plot_height=400, tools=hover.tools())
p.circle(source=df, x="TrainingStartTimeInSeconds", y="FinalObjectiveValue")

show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [11]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))